In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi", 
    "modify.1" : "level4",
    "modify.2" : "level3", 
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
multilevel_lookup.head()

## Now load in the subject data
id = 127
#subjectData = pd.read_csv('./kirby21AllLevels.csv', sep=",")
subjectData = pd.read_csv('https://raw.githubusercontent.com/ds4ph-bme/ds4ph-spring-2022-homework-4-ykim231/main/kirby21AllLevels.csv?token=GHSAT0AAAAAABRIAGHRKHZACQ7FWEF7VHZYYQ2NHJA')
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]
## Merge the subject data with the multilevel data
subjectData = pd.merge(subjectData, multilevel_lookup, on = "roi")
subjectData = subjectData.assign(icv = "ICV")
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))

df = subjectData

## Remove self-looping
#  Add suffix [%level%] to eliminate duplicated node names at different levels
dfsnk = pd.concat(
    [pd.DataFrame({'source': df.icv, 'target': df.level1, 'value':df.volume}),
     pd.DataFrame({'source': df.level1, 'target': df.level2+'[2]', 'value':df.volume}),
     pd.DataFrame({'source': df.level2+'[2]', 'target': df.level3+'[3]', 'value':df.volume}), 
     pd.DataFrame({'source': df.level3+'[3]', 'target': df.level4+'[4]', 'value':df.volume}),
     pd.DataFrame({'source': df.level4+'[4]', 'target': df.roi, 'value':df.volume})],
     axis=0)
    
nodes = np.unique(dfsnk[['source', 'target']], axis=None)
nodes = pd.Series(index=nodes, data=range(len(nodes)))
    
k=go.Figure(
    go.Sankey(
        node={'label': nodes.index},
        link={
            'source': nodes.loc[dfsnk['source']],
            'target': nodes.loc[dfsnk['target']],
            'value': dfsnk['value']},
    ))
k.update_layout(font_size=7, width=1000, height=1500)
k.show() 


homepage link:https://ykim231.github.io/HtmlTestingRepo/

In [ ]:
import sqlite3 as sq3
import pandas as pd

con = sq3.connect('./opioid.db')
sql_pop = pd.read_sql_query("SELECT * from population", con)
sql_ann = pd.read_sql_query("SELECT * from annual", con)
sql_land = pd.read_sql_query("SELECT * from land", con)
con.close

## sqlite> select BUYER_COUNTY, BUYER_STATE, STATE, COUNTY, year, population from population limit 5;
sql_pop[['BUYER_COUNTY', 'BUYER_STATE', 'STATE', 'COUNTY', 'year', 'population']].head(5)

## select * from annual where countyfips = "NA" limit 10;
sql_ann.loc[sql_ann.countyfips=="NA"].head(10)

## sqlite> select * from annual where countyfips = "NA" and BUYER_STATE ! "PR=" limit 10;
sql_ann.loc[(sql_ann.countyfips=="NA")&(sql_ann.BUYER_STATE!="PR")].head(10)

## sqlite> update annual set countyfips = 05097 where BUYER_STATE = "AR" and BUYER_COUNTY = "MONTGOMERY"
sql_ann.countyfips.loc[(sql_ann.BUYER_STATE!="AR")&(sql_ann.BUYER_COUNTY=="MONTGOMERY")] = '05097'

## sqlite> select * from annual where BUYER_STATE = "AR" and BUYER_COUNTY = "MONTGOMERY"
sql_ann.loc[(sql_ann.BUYER_STATE!="AR")&(sql_ann.BUYER_COUNTY=="MONTGOMERY")]

## sqlite> delete from annual where BUYER_COUNTY = "NA"
sql_ann = sql_ann.loc[sql_ann.BUYER_COUNTY!="NA"]

## sqlite> create table land_area as select Areaname, STCOU, LND110210D from land;
land_area = sql_land[['Areaname', 'STCOU', 'LND110210D']]

## sqlite> alter table land_area rename column STCOU to countyfips;
land_area.columns = land_area.columns.str.replace('STCOU', 'countyfips')

#sqlite> create table county_info as select * from population left join land_area using(countyfips);
county_info = pd.merge(sql_pop, land_area, how='left', on='countyfips')

#sqlite> select count(*) from land;
#sqlite> select count(*) from land_area;
#sqlite> select count(*) from county_info;
#sqlite> select count(*) from population;
len(sql_land)
len(land_area)
len(county_info)
len(sql_pop)

28265

In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('https://raw.githubusercontent.com/opencasestudies/ocs-bp-opioid-rural-urban/master/data/simpler_import/county_annual.csv')
df = df.drop(['Unnamed: 0', 'count', 'countyfips'], axis=1)

df2 = df.groupby(['year'], as_index=False).mean() 
j = px.line(df2, x=str('year'), y='DOSAGE_UNIT',
            labels={'year': 'Year',
                    'DOSAGE_UNIT': 'Average number of opioid pills'
                   },
            markers=True
           )
j.update_layout(font_size=10, width=960)
j.show()

figures are published at https://ykim231.github.io/HtmlTestingRepo/